In [18]:
import findspark
findspark.init()

In [19]:
from itertools import chain
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import col, size, create_map, lit, concat, element_at, when, isnan, count
from statistics import mean
from pyspark.sql.types import *

In [20]:
spark = SparkSession.builder.master('local').config('spark.executor.memory', '16g') \
    .config('spark.driver.memory', '16g').appName('Data Preparation and Feature Engineering').getOrCreate()

spark.sparkContext.setLogLevel('ERROR')
sc = spark.sparkContext

In [21]:
schema = StructType([
    StructField("artist_familiarity", DoubleType(), True),
    StructField("artist_hotttnesss", DoubleType(), True),
    StructField("artist_id", StringType(), True),
    StructField("artist_latitude", DoubleType(), True),
    StructField("artist_location", StringType(), True),
    StructField("artist_longitude", DoubleType(), True),
    StructField("artist_name", StringType(), True),
    StructField("release", StringType(), True),
    StructField("song_hotttnesss", DoubleType(), True),
    StructField("title", StringType(), True),
    StructField("danceability", DoubleType(), True),
    StructField("duration", DoubleType(), True),
    StructField("end_of_fade_in", DoubleType(), True),
    StructField("energy", DoubleType(), True),
    StructField("key", LongType(), True),
    StructField("loudness", DoubleType(), True),
    StructField("mode", LongType(), True),
    StructField("start_of_fade_out", DoubleType(), True),
    StructField("tempo", DoubleType(), True),
    StructField("time_signature", LongType(), True),
    StructField("track_id", StringType(), True),
    StructField("year", LongType(), True),
    StructField("artist_terms", ArrayType(StringType()), True),
    StructField("bars_start", ArrayType(DoubleType()), True),
    StructField("beats_start", ArrayType(DoubleType()), True),
    StructField("sections_start", ArrayType(DoubleType()), True),
    StructField("segments_start", ArrayType(DoubleType()), True),
    StructField("tatums_start", ArrayType(DoubleType()), True)])

In [22]:
print('Reading in Data...')
df = spark.read.load('MSD_me.parquet',schema=schema)
df.show(5)

Reading in Data...
+------------------+-------------------+------------------+---------------+---------------+----------------+----------------+--------------------+------------------+----------------+------------+---------+--------------+------+---+--------+----+-----------------+-------+--------------+------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|artist_familiarity|  artist_hotttnesss|         artist_id|artist_latitude|artist_location|artist_longitude|     artist_name|             release|   song_hotttnesss|           title|danceability| duration|end_of_fade_in|energy|key|loudness|mode|start_of_fade_out|  tempo|time_signature|          track_id|year|        artist_terms|          bars_start|         beats_start|      sections_start|      segments_start|        tatums_start|
+------------------+-------------------+------------------+---------------+---------------+----------------

In [23]:


df_Columns = ['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location',
              'artist_longitude',
              'artist_name', 'release', 'song_hotttnesss', 'title', 'danceability', 'duration', 'end_of_fade_in',
              'energy',
              'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'track_id', 'year']

print('Checking for missing values...')
df.select([count(when(col(c).contains('None') | \
                      col(c).contains('NULL') | \
                      (col(c) == '') | \
                      col(c).isNull() | \
                      isnan(c), c
                      )).alias(c) for c in df_Columns]).show()

print('Number of missing artist_terms:', df.count() - df.filter(f.size('artist_terms') > 0).count())
print('Number of missing bars_start:', df.count() - df.filter(f.size('bars_start') > 0).count())
print('Number of missing beats_start:', df.count() - df.filter(f.size('beats_start') > 0).count())
print('Number of missing sections_start:', df.count() - df.filter(f.size('sections_start') > 0).count())
print('Number of missing segments_start:', df.count() - df.filter(f.size('segments_start') > 0).count())
print('Number of missing tatums_start:', df.count() - df.filter(f.size('tatums_start') > 0).count())
print('Number of missing year:', df.count() - df.filter(df['year'] > 0).count())

Checking for missing values...
+------------------+-----------------+---------+---------------+---------------+----------------+-----------+-------+---------------+-----+------------+--------+--------------+------+---+--------+----+-----------------+-----+--------------+--------+----+
|artist_familiarity|artist_hotttnesss|artist_id|artist_latitude|artist_location|artist_longitude|artist_name|release|song_hotttnesss|title|danceability|duration|end_of_fade_in|energy|key|loudness|mode|start_of_fade_out|tempo|time_signature|track_id|year|
+------------------+-----------------+---------+---------------+---------------+----------------+-----------+-------+---------------+-----+------------+--------+--------------+------+---+--------+----+-----------------+-----+--------------+--------+----+
|                 4|                0|        0|           6258|           4292|            6258|          0|      3|           4352|    2|           0|       0|             0|     0|  0|       0|   0|   

In [24]:
print('\nHandling missing values...')
# dropping columns missing ~ half or more of their data
df = df.drop('artist_latitude', 'artist_location', 'artist_longitude', 'song_hotttnesss')

# filtering out rows with missing values
df = df.filter(f.size('artist_terms') > 0).filter(f.size('bars_start') > 0).filter(f.size('beats_start') > 0) \
    .filter(f.size('sections_start') > 0).filter(f.size('tatums_start') > 0)

df = df.dropna()


Handling missing values...


In [25]:
print('\nFeature engineering...')
# computing and adding columns for density, fadedness, variability
df = df.withColumn('density', size(df.segments_start) / df.duration)
df = df.withColumn('fadedness', df.end_of_fade_in + df.duration - df.start_of_fade_out)
df = df.withColumn('variability', size(df.sections_start))


Feature engineering...


In [26]:
# computing and adding column for tonality (key + mode)
dict_mode = {1: 'maj', 0: 'min'}
dict_key = {0: 'DO', 1: 'DO#', 2: 'RE', 3: 'RE#', 4: 'MI', 5: 'FA', 6: 'FA#', 7: 'SOL', 8: 'SOL#', 9: 'LA', 10: 'LA#',
            11: 'SI'}

mapping_expr_key = create_map([lit(x) for x in chain(*dict_key.items())])
mapping_expr_mode = create_map([lit(x) for x in chain(*dict_mode.items())])

df = df.withColumn("key_string", (mapping_expr_key.getItem(col("key"))))
df = df.withColumn('mode_string', (mapping_expr_mode.getItem(col("mode"))))
df = df.withColumn('tonality', concat(df.key_string, lit(' '), df.mode_string))

In [27]:
# computing mean bar and beats starts and adding columns
compute_average = f.udf(lambda x: mean(x))
df = df.withColumn('avg_bars_start', compute_average(df['bars_start']))
df = df.withColumn('avg_beats_start', compute_average(df['beats_start']))

In [28]:
print('\nCalculating statistics on numerical columns...')
df.select('artist_familiarity', 'artist_hotttnesss', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'key', 'loudness',
          'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'year', 'density', 'fadedness', 'variability', 'avg_bars_start', 'avg_beats_start').summary().show()

# drop columns no longer need, including danceability and energy which have mostly 0 values
print('\nDropping empty or unnecessary columns...')
df = df.drop('segments_start', 'tatums_start', 'sections_start', 'bars_start', 'beats_start', 'danceability', 'energy')

df.show()

df.write.save('MSD_w_Year_me.parquet')

final_df = df.drop('year')
final_df.write.save('MSD_FINAL_me.parquet')


Calculating statistics on numerical columns...
+-------+-------------------+-------------------+------------+------------------+------------------+------+-----------------+-------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+--------------------+------------------+------------------+------------------+
|summary| artist_familiarity|  artist_hotttnesss|danceability|          duration|    end_of_fade_in|energy|              key|           loudness|              mode| start_of_fade_out|             tempo|    time_signature|             year|            density|           fadedness|       variability|    avg_bars_start|   avg_beats_start|
+-------+-------------------+-------------------+------------+------------------+------------------+------+-----------------+-------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+-----------------